In [5]:
import xarray as xr
import xclim as xc
from clisops.core.subset import subset_bbox

In [2]:
hist_files = [
    '/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/day/pr/gn/latest/pr_day_ACCESS-ESM1-5_historical_r1i1p1f1_gn_19500101-19991231.nc',
    '/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/day/pr/gn/latest/pr_day_ACCESS-ESM1-5_historical_r1i1p1f1_gn_20000101-20141231.nc',
]

future_files = [
    '/g/data/fs38/publications/CMIP6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/ssp370/r1i1p1f1/day/pr/gn/latest/pr_day_ACCESS-ESM1-5_ssp370_r1i1p1f1_gn_20150101-20641231.nc',
    '/g/data/fs38/publications/CMIP6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/ssp370/r1i1p1f1/day/pr/gn/latest/pr_day_ACCESS-ESM1-5_ssp370_r1i1p1f1_gn_20650101-21001231.nc',   
]

In [3]:
ds_hist = xr.open_mfdataset(hist_files)
ds_hist = subset_bbox(ds_hist, start_date='1995-01-01', end_date='2014-12-31')
ds_hist['pr'] = xc.units.convert_units_to(ds_hist['pr'], 'mm d-1')

In [10]:
ds_future = xr.open_mfdataset(future_files)
ds_future = subset_bbox(ds_future, start_date='2035-01-01', end_date='2064-12-31')
ds_future['pr'] = xc.units.convert_units_to(ds_future['pr'], 'mm d-1')

In [ ]:
qm = sdba.EmpiricalQuantileMapping.train(
    ds_future['pr'],
    ds_hist['pr'],
    nquantiles=100,
    group="time.month",
    kind='*',
)    

To reduce the risk of sharp change in the adjustment at the interface of the months, interp='linear' can be passed to adjust and the adjustment factors will be interpolated linearly. Ex: the factors for the 1st of May will be the average of those for april and those for may.

```python
scen = QM_mo.adjust(sim, extrapolation="constant", interp="linear")
```

See https://xclim.readthedocs.io/en/stable/notebooks/sdba.html#First-example-:-pr-and-frequency-adaptation

```python
sim_ad, pth, dP0 = sdba.processing.adapt_freq(
    pr_ref, pr_sim, thresh="0.05 mm d-1", group="time"
)
```

In [ ]:
qm.ds = qm.ds.assign_coords({'lat': ds_future['lat'], 'lon': ds_future['lon']}) #xclim strips lat/lon attributes
qm.ds = qm.ds.transpose('quantiles', 'month', 'lat', 'lon')